
### AI Model Inference for classification of present of Pneumonia 

#### Load the Libraries for testing our Model with the Group Truth for FDA Approval 

In [4]:
import numpy as np
import pandas as pd
import pydicom
import matplotlib.pyplot as plt
import keras 

import glob
import os
from skimage import data, color
from PIL import Image
from skimage import io
from itertools import chain
from random import sample 
import scipy
import seaborn as sns
from random import sample
from keras.models import load_model
from keras.models import model_from_json
from skimage.transform import rescale, resize, downscale_local_mean

%matplotlib inline

#### Read all DICOM files into a list to visual check 

In [5]:
mydicoms = glob.glob("*.dcm")

In [6]:
dcm1 = pydicom.dcmread(mydicoms[0])
dcm1

(0008, 0016) SOP Class UID                       UI: Secondary Capture Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.3.6.1.4.1.11129.5.5.153751009835107614666834563294684339746480
(0008, 0060) Modality                            CS: 'DX'
(0008, 1030) Study Description                   LO: 'Cardiomegaly'
(0010, 0020) Patient ID                          LO: '1'
(0010, 0040) Patient's Sex                       CS: 'M'
(0010, 1010) Patient's Age                       AS: '58'
(0018, 0015) Body Part Examined                  CS: 'CHEST'
(0018, 5100) Patient Position                    CS: 'AP'
(0020, 000d) Study Instance UID                  UI: 1.3.6.1.4.1.11129.5.5.111396399361969898205364400549799252857604
(0020, 000e) Series Instance UID                 UI: 1.3.6.1.4.1.11129.5.5.195628213694300498946760767481291263511724
(0028, 0002) Samples per Pixel                   US: 1
(0028, 0004) Photometric Interpretation          CS: 'MONOCHROME2'
(0028, 0010) Rows     

### Image Processing 

#### First conduct an image check to confirm the model is considering only the chest x-ray for classification.
This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array of just the imaging data

In [20]:
def check_dicom(filename): 
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)
    if (ds.BodyPartExamined !='CHEST') | (ds.Modality !='DX') | (ds.PatientPosition not in ['PA','AP']):
        print('The model is not valid because the of image position, the image type or the body part are not OK')
        img = None
    else:    
        img = ds.pixel_array
        return img

**This function takes the numpy array output by check_dicom and runs the appropriate pre-processing needed for our model input**

In [21]:
def preprocess_image(img,img_mean,img_std,img_size): 
    img=img/255.0  
    img = (img - img_mean)/img_std
    proc_img = np.resize(img, IMG_SIZE)        
    return proc_img

**Load the model with weights of our trained from the build and train section and compile it.This will make a great impact to confirm how efficient our AI is when compared to the ground Truth from the radiologist** 

In [22]:
def load_model(model_path, weight_path):
    # todo
    json_file = open(model_path, 'r')
    model_json = json_file.read()
    json_file.close()
    model = model_from_json(model_json)
    model.load_weights(weight_path)
    return model

**Predict the model using our best thresholds from FI scores to predict whether or not the image shows the presence of Pneumonia using our trained model. The choice of your thresholds determines how accurate your AI performs.**

In [23]:
def predict_image(model, img, thresh): 
   
    value = model.predict(img)
    if value > thresh:
        print('Pneumonia Present')
    else:
        print("No Pneumonia")

**Finally, lets's test our model with the test DICOM files to classify X-rays images to confirm the presence or absence of Pneumonia**

In [24]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = 'my_model.json'
weight_path = r'xray_class_my_model.best.hdf5'

IMG_SIZE=(1,224,224,3)
img_mean = 0.41373453
img_std = 0.23411134

my_model = load_model(model_path, weight_path)
thresh = 0.54

for i in test_dicoms:
    img = np.array([])
    img = check_dicom(i)
    if img is None:
        continue
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
print(pred)

Load file test1.dcm ...
No Pneumonia
Load file test2.dcm ...
Pneumonia Present
Load file test3.dcm ...
Pneumonia Present
Load file test4.dcm ...
The model is not valid because the of image position, the image type or the body part are not OK
Load file test5.dcm ...
The model is not valid because the of image position, the image type or the body part are not OK
Load file test6.dcm ...
The model is not valid because the of image position, the image type or the body part are not OK
None


#### As can be seen, this model was able to classify the present of pneumonia, the cases with no pneumonia and identify other image types.

### This concludes our Inference for FDA Approval reviews. Please see the FDA approval request PDF file for more details